# Get the data and make it suitable for input

In [11]:
!ls drive/MyDrive/Datasets\ for\ ML/

UTK_dataset.zip


In [14]:
import zipfile

# Specify the correct file path using raw string or double backslashes
zip_file_path = r'drive/MyDrive/Datasets for ML/UTK_dataset.zip'
# or zip_file_path = 'drive/MyDrive/Datasets\\ for\\ ML\\UTK_dataset.zip'

# Unzip the downloaded file
zip_ref = zipfile.ZipFile(zip_file_path)
zip_ref.extractall()
zip_ref.close()

In [15]:
!ls UTKFace/

Streaming output truncated to the last 5000 lines.
 26_0_2_20170116184103544.jpg.chip.jpg	   52_0_3_20170113184200775.jpg.chip.jpg
 26_0_2_20170116184137536.jpg.chip.jpg	   52_0_3_20170113190623929.jpg.chip.jpg
 26_0_2_20170116184500032.jpg.chip.jpg	   52_0_3_20170117190840617.jpg.chip.jpg
 26_0_2_20170116184700065.jpg.chip.jpg	   52_0_3_20170119160357804.jpg.chip.jpg
 26_0_2_20170116184721092.jpg.chip.jpg	   52_0_3_20170119181010188.jpg.chip.jpg
 26_0_2_20170116210411437.jpg.chip.jpg	   52_0_3_20170119200211340.jpg.chip.jpg
 26_0_2_20170116233100347.jpg.chip.jpg	   52_0_3_20170119200241996.jpg.chip.jpg
 26_0_2_20170117172558276.jpg.chip.jpg	   52_0_3_20170119200259595.jpg.chip.jpg
 26_0_2_20170117172618124.jpg.chip.jpg	   52_0_3_20170119200407188.jpg.chip.jpg
 26_0_2_20170117172630501.jpg.chip.jpg	   52_0_3_20170119204331863.jpg.chip.jpg
 26_0_2_20170117172714981.jpg.chip.jpg	   52_0_3_20170119204337838.jpg.chip.jpg
 26_0_2_20170117172733502.jpg.chip.jpg	   52_0_3_20170119205638160.jp

In [17]:
#Get the class names programmatically
import pathlib
import numpy as np

data_dir = pathlib.Path("UTKFace/")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # Created a list of class_names from the subdirectories
#Remove .DS_Store in case it appears
calss_names = class_names[1:] #reset the class names just in case of any abnormalities (extra files)
print(class_names)

['100_0_0_20170112213500903.jpg.chip.jpg'
 '100_0_0_20170112215240346.jpg.chip.jpg'
 '100_1_0_20170110183726390.jpg.chip.jpg' ...
 '9_1_4_20170103200637399.jpg.chip.jpg'
 '9_1_4_20170103200814791.jpg.chip.jpg'
 '9_1_4_20170103213057382.jpg.chip.jpg']


# Build the multi-task learning

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Define the shared base model
def build_shared_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    return model

# Define the binary classification head
def build_binary_classification_head(base_model_output):
    x = layers.Dense(64, activation='relu')(base_model_output)
    binary_classification_output = layers.Dense(1, activation='sigmoid', name='binary_classification_output')(x)
    return binary_classification_output

# Define the regression head
def build_regression_head(base_model_output):
    x = layers.Dense(64, activation='relu')(base_model_output)
    regression_output = layers.Dense(1, name='regression_output')(x)
    return regression_output

# Input shape for the image
input_shape = (128, 128, 3)

# Number of classes for binary classification
num_classes_binary = 1

# Shared base model
base_model = build_shared_model(input_shape)

# Binary classification head
binary_classification_output = build_binary_classification_head(base_model.output)

# Regression head
regression_output = build_regression_head(base_model.output)

# Create the multi-task model
model = models.Model(inputs=base_model.input, outputs=[binary_classification_output, regression_output])

# Compile the model
model.compile(optimizer=Adam(),
              loss={'binary_classification_output': 'binary_crossentropy', 'regression_output': 'rmse'},
              metrics={'binary_classification_output': 'accuracy', 'regression_output': 'mse'})

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 conv2d_input (InputLayer)   [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 126, 126, 32)         896       ['conv2d_input[0][0]']        
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 61, 61, 64)           18496     ['max_pooling2d[0][0]']   

# Train the model

In [ ]:
# Assuming you have X_train (input images), y_class (classification labels), and y_reg (regression labels)

# Convert classification labels to one-hot encoding
y_class_one_hot = tf.keras.utils.to_categorical(y_class, num_classes=num_classes)

# Train the model using the fit method
epochs = 10
batch_size = 32

history = model.fit(
    X_train,
    {'classification_output': y_class_one_hot, 'regression_output': y_reg},
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2  # Adjust the validation split as needed
)

# Save the trained model if needed
model.save('multi_task_model.h5')

# Check for overfitting

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(12, 6))

# Plot Classification Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['classification_output_loss'], label='classification_loss')
plt.plot(history.history['val_classification_output_loss'], label='val_classification_loss')
plt.title('Classification Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Regression Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['regression_output_loss'], label='regression_loss')
plt.plot(history.history['val_regression_output_loss'], label='val_regression_loss')
plt.title('Regression Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()